In [13]:
import csv
from datetime import datetime


In [14]:
# Transaction details in dictionary format
transaction_counter = 0

def process_transaction(transaction):
    global transaction_counter
    transaction_counter += 1
    transaction['transaction_id'] = transaction_counter
    return transaction

    


In [15]:
def load_transactions(file_path):
    transactions = []
    global transaction_counter
    transaction_counter = 0
    
    try:
        with open(file_path, mode='r',) as file:
            reader = csv.DictReader(file)
            next(reader)  # Skip header row
            for row in reader:
                try:
                    date = datetime.strptime(row['date'], '%Y-%m-%d')
                    row['date'] = date.strftime('%Y-%m-%d')
                    amount = float(row['amount'])
                    row['amount'] = amount
                    transactiontype = row['transaction_type'].lower()
                    if transactiontype not in ['credit', 'debit']:
                        raise ValueError(f"Invalid transaction type: {transactiontype}")
                    row['transaction_type'] = transactiontype
                    transaction = {
                        'date': row['date'],
                        'amount': row['amount'],
                        'transaction_type': row['transaction_type'],
                        'description': row.get('description', ''),
                        'transaction_id': None
                    }
                    transactions.append(process_transaction(row))
                except Exception as e:
                    print(f"Error processing row: {row}. Error: {e}")
    except Exception as e:
        print(f"Error opening file: {file_path}. Error: {e}")

In [22]:
def add_transaction(transactions, transaction):
    try:
        date = datetime.strptime(transaction['date'], '%Y-%m-%d')
        transaction['date'] = date.strftime('%Y-%m-%d')
        transactions.append(process_transaction(transaction))
    except Exception as e:
        print(f"Error adding transaction: {transaction}. Error: {e}")


In [ ]:
def update_transaction(transactions, transaction_id, updated_transaction):
    for transaction in transactions:
        if transaction['transaction_id'] == transaction_id:
            try:
                date = datetime.strptime(updated_transaction['date'], '%Y-%m-%d')
                updated_transaction['date'] = date.strftime('%Y-%m-%d')
                transaction.update(updated_transaction)
                return
            except Exception as e:
                print(f"Error updating transaction ID {transaction_id}: {e}")
    print(f"Transaction ID {transaction_id} not found.")

In [ ]:
def delete_transaction(transactions, transaction_id):
    global transaction_counter
    for i, transaction in enumerate(transactions):
        if transaction['transaction_id'] == transaction_id:
            del transactions[i]
            transaction_counter -= 1
            return
    print(f"Transaction ID {transaction_id} not found.")


In [ ]:
def analyze_transactions(transactions):
    total_credit = sum(t['amount'] for t in transactions if t['transaction_type'] == 'credit')
    total_debit = sum(t['amount'] for t in transactions if t['transaction_type'] == 'debit')
    balance = total_credit - total_debit
    return {
        'total_credit': total_credit,
        'total_debit': total_debit,
        'balance': balance
    }


In [ ]:
def save_transactions(transactions, file_path):
    try:
        with open(file_path, mode='w', newline='') as file:
            fieldnames = ['transaction_id', 'date', 'amount', 'transaction_type', 'description']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for transaction in transactions:
                writer.writerow(transaction)
    except Exception as e:
        print(f"Error saving transactions to file: {file_path}. Error: {e}")

In [ ]:
def generate_report(transactions):
    report = analyze_transactions(transactions)
    print("Transaction Report:")
    print(f"Total Credit: {report['total_credit']}")
    print(f"Total Debit: {report['total_debit']}")
    print(f"Balance: {report['balance']}")
    return report